In [1]:
!pip install -qU langchain-community wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 19.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmanager 0.7.1 requires sqlalchemy==1.2.19, but you have sqlalchemy 2.0.45 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.2.7 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.1.0 which is incompatible.


In [2]:
import re
from collections import Counter
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

In [3]:
def ExtractContentFromWikipedia(query="Nigerian History"):
    txt = ""
    for docs in retriever.invoke("Nigerian History"):
        page_content = docs.page_content
        txt+= " " + page_content
    return txt.lower()

extracted_txt = ExtractContentFromWikipedia()

In [4]:
init_word_list = re.findall(pattern=r"\w+|[^\s\w]+", string=extracted_txt, flags=re.MULTILINE)

init_word_list[:5]

['the', 'history', 'of', 'nigeria', 'can']

In [5]:
word_count = Counter(init_word_list) # Returns a dictionary of word and its counts

word_count.most_common(10)

[(',', 142),
 ('the', 126),
 ('.', 114),
 ('of', 83),
 ('in', 68),
 ('and', 62),
 ('-', 47),
 ('nigeria', 32),
 ('to', 29),
 ('west', 29)]

In [6]:
eos_token = "</end>"

initial_corpus_representation = {}
for word in word_count:
    wrd_split = list(word)+[eos_token]
    initial_corpus_representation[word] = wrd_split

#### OUTPUT #### 
# {'beginning': ['b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g', '</w>']}

In [7]:
init_tokens = set()
for word in initial_corpus_representation:
    wrd_lst = initial_corpus_representation[word]
    init_tokens.update(wrd_lst)

#### OUTPUT ####
# Initial vocabulary created with 31 unique symbols.
# Initial vocabulary symbols: ["'", '(', ')', ',', '-', '.', ':', '</end>', '&', ...]